In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [26.7 kB]
Hit:12 http://ppa.launchpad.ne

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon-Challenge-Vine-Review").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df = vine_df.withColumn('star_rating',vine_df.star_rating.cast('integer')) #convert star_rating column from string to integer
vine_df.show()
vine_df.printSchema()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [7]:
# filter by products with 20 or more total votes 
vine_votes_df = vine_df.filter('total_votes>=20')
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
|R39BVCCVPRV6F5|          5|           10|         22|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
|R1ZG19X1NOLCRF|          5|            5|         21|   N|                Y|
|R1YOX5Z4GF3ZSW|          1|            1|         22|   N|                Y|
| R2DHAM7J1KMWN|          5|           23|         24|   N|     

In [8]:
# filter by products with 50% or more helpful votes
vine_helpful_df = vine_votes_df.withColumn('helpful_mix', vine_votes_df['helpful_votes']/vine_votes_df['total_votes'])
vine_helpful_df.show()
vine_help50_df = vine_helpful_df.filter('helpful_mix>0.50')
vine_help50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         helpful_mix|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|  0.8695652173913043|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|  0.9019607843137255|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|                 1.0|
|R39BVCCVPRV6F5|          5|           10|         22|   N|                Y| 0.45454545454545453|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|  0.9722222222222222|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|  0.8666666666666667|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|  0.9545454545454546|
|R1ZG19X1N

In [12]:
# filter vine program reviews
vine_yes_df = vine_help50_df.filter(vine_help50_df['vine'] == 'Y')
vine_yes_df.show()
vine_yes_length = vine_yes_df.count()
print(vine_yes_length)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_mix|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3QIQRG107AP19|          5|           38|         41|   Y|                N| 0.926829268292683|
|R1ZNBHGCYE8VN1|          5|          175|        181|   Y|                N|0.9668508287292817|
|R1BNZJ9IKGRBC2|          4|           63|         70|   Y|                N|               0.9|
|R2IXTN4PBRHBO2|          5|           19|         24|   Y|                N|0.7916666666666666|
|R32AOW3L09SYMQ|          5|           29|         34|   Y|                N|0.8529411764705882|
|R34Z4VTLF7PC6X|          4|           33|         36|   Y|                N|0.9166666666666666|
|R25403NE9JCRZZ|          2|           36|         40|   Y|                N|               0.9|
|R2QT2G3YRGC0PS|          5|  

In [14]:
# filter non-vine program reviews
vine_no_df = vine_help50_df.filter(vine_help50_df['vine'] == 'N')
vine_no_df.show()
vine_no_length = vine_no_df.count()
print(vine_no_length)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_mix|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|0.8695652173913043|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|0.9019607843137255|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|               1.0|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|0.9722222222222222|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|0.8666666666666667|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|0.9545454545454546|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|0.9583333333333334|
|R26EZ2INBETPU2|          1|  

In [18]:
# total reviews, 5-star reviews, and % of 5-star reviews for vine program
vine_y_stats_df = vine_yes_df.groupBy("star_rating").count()
vine_y_stats_df.show()
vine_y_filter_df = vine_y_stats_df.filter(vine_y_stats_df['star_rating'] == 5)
vine_y_filter_df.show()
vine_y_summ_df = vine_y_filter_df.withColumn('5_rating_ppt', vine_y_filter_df['count']/vine_yes_length)
vine_y_summ_df.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|    7|
|          3|   33|
|          5|  163|
|          4|   74|
|          2|    8|
+-----------+-----+

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          5|  163|
+-----------+-----+

+-----------+-----+------------------+
|star_rating|count|      5_rating_ppt|
+-----------+-----+------------------+
|          5|  163|0.5719298245614035|
+-----------+-----+------------------+



In [19]:
# total reviews, 5-star reviews, and % of 5-star reviews for non-vine program
vine_n_stats_df = vine_no_df.groupBy("star_rating").count()
vine_n_stats_df.show()
vine_n_filter_df = vine_n_stats_df.filter(vine_n_stats_df['star_rating'] == 5)
vine_n_filter_df.show()
vine_n_summ_df = vine_n_filter_df.withColumn('5_rating_ppt', vine_n_filter_df['count']/vine_no_length)
vine_n_summ_df.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 5370|
|          3| 3452|
|          5|14603|
|          4| 5599|
|          2| 2428|
+-----------+-----+

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          5|14603|
+-----------+-----+

+-----------+-----+-------------------+
|star_rating|count|       5_rating_ppt|
+-----------+-----+-------------------+
|          5|14603|0.46429479842299376|
+-----------+-----+-------------------+

